# plot_theta_q_profiles.ipynb

This code plots vertical profiles of potential temperature and water vapour from sondes, initialisation fields, and model output.

In [ ]:
import xarray as xr
import pandas as pn
import numpy as np
import numpy.ma as ma
import datetime as dt
import matplotlib.pyplot as plt
import capricorn_figure_props as fprops
import capricorn_functions as cfn
import capricorn_constants as const
from simulations import simulations
from sonde import Sonde
from glob import glob
from importlib import reload
reload(cfn)
reload(const)


In [ ]:
def hybrid_height(level_height, sigma, surface_altitude):
    '''
    Calculate the UM hybrid height coordinate from level height, sigma, and surface altitude fields
    '''
    z = level_height + sigma*surface_altitude
    return z


## Load processed model fields
Met Office analysis, ERA5, and model output has been processed in capricorn-collocate-init.py

In [ ]:
# UKMO
ukmo_pot_temp = xr.open_dataset('data/model/UKMO_init_theta_cap.nc')
ukmo_humidity = xr.open_dataset('data/model/UKMO_init_humidity_cap.nc')

# ERA5
era_pot_temp = xr.open_dataset('data/model/ERA5_init_theta_cap.nc')
era_humidity = xr.open_dataset('data/model/ERA5_init_humidity_cap.nc')

# model
fig_simulations = simulations[[0,4]]
model_pot_temp = [xr.open_dataset(f'data/model/{sim.code}_init_theta_cap.nc')
                  for sim in fig_simulations]
model_humidity = [xr.open_dataset(f'data/model/{sim.code}_init_humidity_cap.nc')
                  for sim in fig_simulations]


## Load radiosonde data from file

In [ ]:
# select sonde files using dates in filenames
dates = [
    '20180131',
    '20180201',
    '20180202',
    '20180203',
    '20180204',
]
sonde_dir = '../polarres-obs/data/capricorn2/sondes/'
sonde_files = [fn for fn in glob(sonde_dir+'*.nc') if fn.split('_')[4] in dates]
sondes = []
for f,fn in enumerate(np.sort(sonde_files)):
    print(f'[INFO] loading sonde [{f+1}/{len(sonde_files)}]')
    sonde = Sonde(fn)
    sonde.load_temp()
    sonde.load_humidity()
    sonde.load_altitude()
    sondes.append(sonde)
sondes = np.array(sondes)

# get sondes released at times closest to target model initialisation times
fmt = '%Y%m%d'
dates_dt = np.array([dt.datetime.strptime(T,fmt) for T in dates])
i_Tsonde_to_Tmodel = [np.argmin([abs((sonde.release_time-date).total_seconds())
                                for sonde in sondes]) for date in dates_dt]


## Plot figure 6

In [ ]:
colours = [sim.colour for sim in fig_simulations]
mosaic = """
    aabbcc
    .ddee.
    """
keys = ['a','b','c','d','e']
figsize = (20*fprops.in_cm,20*fprops.in_cm)
fig_simulations = simulations[[0,4]]
fig, axes = plt.subplot_mosaic(mosaic, figsize=figsize, dpi=fprops.dpi)
zmax = 1500  # maximum height plotted in m
for d, date in enumerate(dates_dt):
    print(date)
    ax = axes[keys[d]]
    sonde = sondes[i_Tsonde_to_Tmodel][d]
    sonde_z = sonde.altitude
    ksonde = np.where(sonde_z<zmax)[0]
    ax.plot(sonde.pot_temp[ksonde], sonde_z[ksonde],
            'k-', label='Sonde', lw=1.5, zorder=6)
    ukmo_z = hybrid_height(
        ukmo_pot_temp.level_height,
        ukmo_pot_temp.sigma,
        ukmo_pot_temp.surface_altitude
    )
    da = ukmo_pot_temp.air_potential_temperature
    ax.plot(da.sel(time=date).where(ukmo_z<zmax).values-273.15,
            ukmo_z, label='UKMO analysis', color=colours[0], lw=1.5)
    da = era_pot_temp.pot_temp
    era_z = era_pot_temp.altitude.sel(time=date)
    ax.plot(da.sel(time=date).where(era_z<zmax).values-273.15,
            era_z, label='ERA5', color=colours[1], lw=1.5)
    for s,sim in enumerate(fig_simulations):
        model_z = model_pot_temp[s].TH_1_60_eta_theta*const.ztop
        da = model_pot_temp[s].STASH_m01s00i004
        ax.plot(da.sel(T1HR=date, method='nearest').where(model_z<zmax)-273.15, model_z,
                label=sim.label, c=sim.colour, lw=1)
    ax.set_title(f'({keys[d]})', loc='left', fontsize=fprops.label_fs)
    ax.set_xlabel(r'$\theta$ [$\degree$C]',fontsize=fprops.ax_label_fs)
    ax.tick_params(axis='x', labelsize=fprops.ax_fs)
    ax.tick_params(axis='y', labelsize=fprops.ax_fs)
axes[keys[0]].legend(fontsize=fprops.legend_fs)
for x in ['a','d']:
    axes[x].set_ylabel('Altitude [m]', fontsize=fprops.ax_label_fs)
for x in ['b','c','e']:
    axes[x].set_yticklabels([])
fig.subplots_adjust(hspace=0.3)
plt.savefig('figures/figure6.png', bbox_inches='tight', facecolor='white')



## Plot figure S7

In [ ]:
colours = [sim.colour for sim in fig_simulations]
mosaic = """
    aabbcc
    .ddee.
    """
keys = ['a','b','c','d','e']
figsize = (20*fprops.in_cm,20*fprops.in_cm)
fig_simulations = simulations[[0,4]]
fig, axes = plt.subplot_mosaic(mosaic, figsize=figsize, dpi=fprops.dpi)
zmax = 1500  # maximum height plotted in m
for d, date in enumerate(dates_dt):
    print(date)
    ax = axes[keys[d]]
    sonde = sondes[i_Tsonde_to_Tmodel][d]
    sonde_z = sonde.altitude
    ksonde = np.where(sonde_z<zmax)[0]
    ax.plot(sonde.humidity[ksonde], sonde_z[ksonde],
            'k-', label='Sonde', lw=1.5, zorder=6)
    ukmo_z = hybrid_height(
        ukmo_humidity.level_height,
        ukmo_humidity.sigma,
        ukmo_humidity.surface_altitude
    )
    da = ukmo_humidity.specific_humidity*1000
    ax.plot(da.sel(time=date).where(ukmo_z<zmax).values,
            ukmo_z, label='UKMO analysis', color=colours[0], lw=1.5)
    da = era_humidity.q*1000
    era_z = era_humidity.altitude.isel(time=d)
    ax.plot(da.sel(time=date).where(era_z<zmax).values,
            era_z, label='ERA5', color=colours[1], lw=1.5)
    for s,sim in enumerate(fig_simulations):
        model_z = model_humidity[s].TH_1_60_eta_theta*const.ztop
        da = model_humidity[s].STASH_m01s00i010*1000
        ax.plot(da.sel(T1HR=date, method='nearest').where(model_z<zmax), model_z,
                label=sim.label, c=sim.colour, lw=1)
    ax.set_title(f'({keys[d]})', loc='left', fontsize=fprops.label_fs)
    ax.set_xlabel('q$_v$ [g kg-1]',fontsize=fprops.ax_label_fs)
    ax.tick_params(axis='x', labelsize=fprops.ax_fs)
    ax.tick_params(axis='y', labelsize=fprops.ax_fs)
axes[keys[0]].legend(fontsize=fprops.legend_fs)
for x in ['a','d']:
    axes[x].set_ylabel('Altitude [m]', fontsize=fprops.ax_label_fs)
for x in ['b','c','e']:
    axes[x].set_yticklabels([])
fig.subplots_adjust(hspace=0.3)
plt.savefig('figures/figureS7.png', bbox_inches='tight', facecolor='white')



## Plot figure S8

In [ ]:
fig = plt.figure(figsize=(10*fprops.in_cm,10*fprops.in_cm), dpi=fprops.dpi)
fig_simulations = simulations[[0,4,5]]
for s,sim in enumerate(fig_simulations):
    zbl = xr.open_dataarray(f'data/model/{sim.code}_m01s00i025_CAP.nc')
    zbl_bins, zbl_stats = cfn.get_hist_bins(zbl)
    zbl_cts, zbl_bins = np.histogram(zbl, bins=zbl_bins, density=True)
    plt.stairs(zbl_cts, zbl_bins, color=sim.colour, fill=True,
               alpha=0.2, zorder=1, lw=sim.linewidth)
    plt.stairs(zbl_cts, zbl_bins, color=sim.colour, fill=False,
               zorder=1, label=sim.label, lw=sim.linewidth)
    ax = fig.gca()
    plt.text(x=0.95, y=0.85-s*0.1, s=f'{zbl_stats[2]:.2f} m', ha='right',
            va='top', color=sim.colour, transform=ax.transAxes)
plt.xticks(fontsize=fprops.ax_fs)
plt.yticks(fontsize=fprops.ax_fs)
plt.xlabel('z$_{{BL}}$ [m]', fontsize=fprops.ax_label_fs)
plt.ylabel('PDF', fontsize=fprops.ax_label_fs)
plt.legend(fontsize=fprops.legend_fs)
plt.savefig('figures/figureS8.png', bbox_inches='tight',
            facecolor='white')


## Plot profiles for ocean domain

In [ ]:
dates_ocean = [
    '20180122',
    '20180123',
    '20180124',
    '20180125',
    '20180126',
    '20180127',
]

sonde_dir = '../polarres-obs/data/capricorn2/sondes/'
sonde_files = [fn for fn in glob(sonde_dir+'*.nc') if fn.split('_')[4] in dates_ocean]
sondes = []
for f,fn in enumerate(np.sort(sonde_files)):
    print(f'[INFO] loading sonde [{f+1}/{len(sonde_files)}]')
    sonde = Sonde(fn)
    sonde.load_temp()
    sonde.load_humidity()
    sonde.load_altitude()
    sondes.append(sonde)
sondes = np.array(sondes)
# get sondes released at times closest to target model initialisation times
fmt = '%Y%m%d'
dates_dt = np.array([dt.datetime.strptime(T,fmt) for T in dates_ocean])
i_Tsonde_to_Tmodel = [np.argmin([abs((sonde.release_time-date).total_seconds())
                                for sonde in sondes]) for date in dates_dt]

# model
sim = simulations[1]
model_pot_temp = xr.open_dataset(f'data/model/{sim.code}_init_theta_cap.nc')
model_humidity = xr.open_dataset(f'data/model/{sim.code}_init_humidity_cap.nc')

### theta profiles from UKMO, ERA5, and CONTROL for ocean domain

In [ ]:
keys = ['a','b','c','d','e','f']
figsize = (20*fprops.in_cm,20*fprops.in_cm)
fig, axes = plt.subplots(2, 3, figsize=figsize, dpi=fprops.dpi)
zmax = 1500  # maximum height plotted in m
for d, date in enumerate(dates_dt):
    print(date)
    ax = axes.flatten()[d]
    sonde = sondes[i_Tsonde_to_Tmodel][d]
    sonde_z = sonde.altitude
    ksonde = np.where(sonde_z<zmax)[0]
    ax.plot(sonde.pot_temp[ksonde], sonde_z[ksonde],
            'k-', label='Sonde', lw=1.5, zorder=6)
    ukmo_z = hybrid_height(
        ukmo_pot_temp.level_height,
        ukmo_pot_temp.sigma,
        ukmo_pot_temp.surface_altitude
    )
    da = ukmo_pot_temp.air_potential_temperature
    ax.plot(da.sel(time=date).where(ukmo_z<zmax).values-273.15,
            ukmo_z, label='UKMO analysis', color=colours[0], lw=1.5)
    da = era_pot_temp.pot_temp
    era_z = era_pot_temp.altitude.sel(time=date)
    ax.plot(da.sel(time=date).where(era_z<zmax).values-273.15,
            era_z, label='ERA5', color=colours[1], lw=1.5)
    model_z = model_pot_temp.TH_1_60_eta_theta*const.ztop
    da = model_pot_temp.STASH_m01s00i004
    ax.plot(da.sel(T1HR=date, method='nearest').where(model_z<zmax)-273.15, model_z,
            label=sim.label, c=sim.colour, lw=1)
    ax.set_title(f'({keys[d]})', loc='left', fontsize=fprops.label_fs)
    ax.set_xlabel(r'$\theta$ [$\degree$C]',fontsize=fprops.ax_label_fs)
    ax.tick_params(axis='x', labelsize=fprops.ax_fs)
    ax.tick_params(axis='y', labelsize=fprops.ax_fs)
axes[0,0].legend(fontsize=fprops.legend_fs)
for ax in axes[:,0]:
    ax.set_ylabel('Altitude [m]', fontsize=fprops.ax_label_fs)
for ax in axes[:,2:].flatten():
    ax.set_yticklabels([])
fig.subplots_adjust(hspace=0.3)
plt.savefig(f'figures/T_profiles_{sim.code}.png', bbox_inches='tight', facecolor='white')

### q profiles from UKMO, ERA5, and CONTROL for ocean domain

In [ ]:
keys = ['a','b','c','d','e','f']
figsize = (20*fprops.in_cm,20*fprops.in_cm)
fig, axes = plt.subplots(2, 3, figsize=figsize, dpi=fprops.dpi)
zmax = 1500  # maximum height plotted in m
for d, date in enumerate(dates_dt):
    print(date)
    ax = axes.flatten()[d]
    sonde = sondes[i_Tsonde_to_Tmodel][d]
    sonde_z = sonde.altitude
    ksonde = np.where(sonde_z<zmax)[0]
    ax.plot(sonde.humidity[ksonde], sonde_z[ksonde],
            'k-', label='Sonde', lw=1.5, zorder=6)
    ukmo_z = hybrid_height(
        ukmo_humidity.level_height,
        ukmo_humidity.sigma,
        ukmo_humidity.surface_altitude
    )
    da = ukmo_humidity.specific_humidity*1e3
    ax.plot(da.sel(time=date).where(ukmo_z<zmax).values,
            ukmo_z, label='UKMO analysis', color=colours[0], lw=1.5)
    da = era_humidity.q*1e3
    era_z = era_humidity.altitude.sel(time=date)
    ax.plot(da.sel(time=date).where(era_z<zmax).values,
            era_z, label='ERA5', color=colours[1], lw=1.5)
    model_z = model_humidity.TH_1_60_eta_theta*const.ztop
    da = model_humidity.STASH_m01s00i010*1e3
    ax.plot(da.sel(T1HR=date, method='nearest').where(model_z<zmax), model_z,
            label=sim.label, c=sim.colour, lw=1)
    ax.set_title(f'({keys[d]})', loc='left', fontsize=fprops.label_fs)
    ax.set_xlabel(r'q$_v$ [g kg-1]',fontsize=fprops.ax_label_fs)
    ax.tick_params(axis='x', labelsize=fprops.ax_fs)
    ax.tick_params(axis='y', labelsize=fprops.ax_fs)
axes[0,0].legend(fontsize=fprops.legend_fs)
for ax in axes[:,0]:
    ax.set_ylabel('Altitude [m]', fontsize=fprops.ax_label_fs)
for ax in axes[:,2:].flatten():
    ax.set_yticklabels([])
fig.subplots_adjust(hspace=0.3)
plt.savefig(f'figures/q_profiles_{sim.code}.png', bbox_inches='tight', facecolor='white')